<a href="https://colab.research.google.com/github/shilz1007/shilz1007/blob/main/Proj_ResNet_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import copy
import time
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from pandas import io 
from torch.utils.data import Dataset,DataLoader
from torchvision.io import read_image
import matplotlib.pyplot as plt
import skimage.io
from PIL import Image
from PIL import Image, ImageDraw
import random
%matplotlib inline

In [2]:
path1 = '/content/drive/MyDrive/SCD/Validation/GroundTruth.csv'
path2 = '/content/drive/MyDrive/SCD/Images'

In [3]:
df = pd.read_csv(path1)
rows = len(df.index)
rows

10015

In [4]:
for ind in range(rows):
  df['image'][ind] = df['image'][ind] +'.jpg'
  #print(df['image'][ind])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
df.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310.jpg,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
class SkinData(Dataset):
  def __init__(self,csv_file,root_dir,transform=None):
    self.annotations = csv_file
    self.root_dir = root_dir
    self.transform = transform
  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
  
    img_path = os.path.join(self.root_dir,self.annotations.iloc[index,0])
     #image = skimage.io.imread(img_path)
    image_new = read_image(img_path)
    y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

    #if self.transform:
    #   image_new = self.transform(image_new,y_label)

    return (image_new,y_label)  


     


 

In [9]:
## hyper parameters
import numpy as np

in_channel = 3
num_classes = 2
learning_rate = 0.001
b_size = 100
num_epochs = 25

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
      transforms.Compose([
      transforms.RandomResizedCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean, std)                          
    ]),
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load data
## load data 
#csv_file = '/content/drive/MyDrive/GroundTruth.csv'
dataset =  SkinData(csv_file = df,
                    root_dir= '/content/drive/MyDrive/SCD/Images',
                    transform=data_transforms) 

data_size = (len(dataset)) 

train_set ,test_set = torch.utils.data.random_split(dataset,[7011,3004])



train_loader = DataLoader(train_set, batch_size = b_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size = b_size, shuffle=True)

#-------------------------------------------
#train_features, train_labels = next(iter(train_loader))
#print(f"Feature batch shape: {train_features.size()}")
#print(f"Labels batch shape: {train_labels.size()}")
#img = train_features[0].squeeze()
#label = train_labels[0]
#plt.imshow(img)
#plt.show()
#print(f"Label: {label}")


In [10]:
model = torchvision.models.resnet18(pretrained=True)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=learning_rate,momentum = 0.9)

for epoch in range(num_epochs):
  losses = []

  for batch_idx, (data,targets) in enumerate(train_loader):
    data = data.to(device)
    targets = targets.to(device)

    data = data.float()
    #print(model(data).dtype)
    scores = model(data)
    loss = criterion(scores,targets)

    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

  print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            x = x.float()
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)    




Cost at epoch 0 is 1.0521297878782514
Cost at epoch 1 is 0.22911974252529546
Cost at epoch 2 is 0.19656149193014896
Cost at epoch 3 is 0.17649881142965504
Cost at epoch 4 is 0.15846921681938037
Cost at epoch 5 is 0.12496518752944301
Cost at epoch 6 is 0.1047818351169707
Cost at epoch 7 is 0.08150851495668922
Cost at epoch 8 is 0.06626399729768155
Cost at epoch 9 is 0.054941877853912365
Cost at epoch 10 is 0.03950915915865294
Cost at epoch 11 is 0.02775837749269017
Cost at epoch 12 is 0.019895119238382494
Cost at epoch 13 is 0.016221077076461112
Cost at epoch 14 is 0.014402508735656738
Cost at epoch 15 is 0.011093971109621122
Cost at epoch 16 is 0.010120001608427142
Cost at epoch 17 is 0.006349049787312536
Cost at epoch 18 is 0.0066884295870615566
Cost at epoch 19 is 0.010804476196759604
Cost at epoch 20 is 0.004783244908485614
Cost at epoch 21 is 0.004072885196672385
Cost at epoch 22 is 0.002959247517832358
Cost at epoch 23 is 0.003731299986944757
Cost at epoch 24 is 0.0030419685882778